In [1]:
import sklearn
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow import keras

2024-02-29 13:45:48.299810: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-29 13:45:48.302352: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-29 13:45:48.337955: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 13:45:48.337987: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 13:45:48.338966: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
pd.set_option("display.max_columns", 20)
pd.set_option("display.max_rows", 300)
pd.set_option("display.precision", 5)
pd.set_option("display.max_seq_items", 50)
pd.set_option("display.max_seq_items", 50)

In [3]:
keras.backend.clear_session()

1. load data

In [7]:
labels_path = (
    "/home/t.afanasyeva/MAT_rnaseq/gene_counts/20240224_labels_138_samples.csv"
)
data_path = (
    "/home/t.afanasyeva/MAT_rnaseq/gene_counts/20240224_training_data_138_samples.csv"
)

In [55]:
data = pd.read_csv(data_path, index_col="Unnamed: 0")
labels = pd.read_csv(labels_path, index_col="Unnamed: 0")

In [56]:
data = data.drop(labels="MD215_7115_Pam3_2", axis=0)
labels = labels.drop(labels="MD215_7115_Pam3_2", axis=0)

In [19]:
data.shape, labels.shape

((137, 47698), (137, 1))

In [20]:
# data.describe()

1.3 Add Feature selection

In [21]:
# from sklearn.neighbors import LocalOutlierFactor

# clf = LocalOutlierFactor(n_neighbors=4)
# outliers = clf.fit_predict(data)
# mask = [outlier != -1 for outlier in outliers]
# features = data[mask]
# labels = labels[mask]

In [57]:
data = data.reset_index(drop=True)
labels = labels.reset_index(drop=True)

1.1. Add SMOT

In [58]:
data.shape

(137, 47698)

In [59]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(sampling_strategy="not majority", k_neighbors=1)
X_train, y_train = sm.fit_resample(data, labels)

In [60]:
X_train.shape

(186, 47698)

In [61]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.2, random_state=0, shuffle=True, stratify=labels
)

In [62]:
from sklearn.base import BaseEstimator, TransformerMixin


class LibraryLengthNormalizer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # Normalise the gene counts to the library size
        # X = X.T.apply(lambda x: (x / (x.sum() if x.sum() != 0 else 1)) * 1000000) # Code when made the graphs
        # return X.T

        X = X.apply(lambda x: (x / (x.sum() if x.sum() != 0 else 1)) * 1000000, axis=1)
        return X

In [63]:
import time
from sklearn.pipeline import Pipeline
from feature_engine.selection import (
    DropDuplicateFeatures,
    DropConstantFeatures,
)
from feature_engine.outliers import Winsorizer
from sklearn.preprocessing import PowerTransformer

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectKBest, chi2, SelectFromModel

forest = ExtraTreesClassifier(n_estimators=250, max_depth=5, random_state=1)


feature_transformer = Pipeline(
    [
        (
            "drop_duplicates",
            DropDuplicateFeatures(),
        ),
        ("drop_constant_feature", DropConstantFeatures(tol=0.50)),
        ("normalise_for_library_size", LibraryLengthNormalizer()),
        ("cap_outliers", Winsorizer(capping_method="gaussian", tail="right", fold=3)),
    ]
)

feature_selector = Pipeline(
    [
        ("select_k_best", SelectKBest(chi2, k=1000)),
        (
            "trasnform_and_scale",
            PowerTransformer(method="yeo-johnson", standardize=True),
        ),
        ("select_forest", SelectFromModel(forest, threshold=0.001)),
    ]
)

print("Number of features before: ", X_train.shape[1])
start_time = time.time()


X_train = feature_transformer.fit_transform(X_train)
X_test = feature_transformer.transform(X_test)

feature_selector.fit(X_train, y_train)
X_train = feature_selector.transform(X_train)
X_test = feature_selector.transform(X_test)

end_time = time.time()
print("Number of features after: ", X_train.shape[1])
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")

Number of features before:  47698


/home/t.afanasyeva/miniforge3/envs/MATseq/lib/python3.10/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Number of features after:  326
Elapsed time: 37.77 seconds


1.3 Prepare data for each model

In [31]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()

# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

In [64]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

le = LabelEncoder()

y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

/home/t.afanasyeva/miniforge3/envs/MATseq/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/t.afanasyeva/miniforge3/envs/MATseq/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [65]:
y_train_nn = to_categorical(y_train)
y_test_nn = to_categorical(y_test)
num_classes = y_train_nn.shape[1]
num_classes

6

2. Initialise Random forest, XGBClassifier, MLPC, NN

In [33]:
from sklearn.svm import SVC, LinearSVC

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier, ExtraTreesClassifier

In [34]:
# Defines the best parameters for MLPC
params = {
    "activation": "relu",
    "alpha": 0.0001,
    "batch_size": "auto",
    "beta_1": 0.9,
    "beta_2": 0.999,
    "early_stopping": False,
    "epsilon": 1e-08,
    "hidden_layer_sizes": (100,),
    "learning_rate": "constant",
    "learning_rate_init": 0.001,
    "max_fun": 15000,
    "max_iter": 600,
    "momentum": 0.9,
    "n_iter_no_change": 10,
    "nesterovs_momentum": True,
    "power_t": 0.5,
    "random_state": 0,
    "shuffle": True,
    "solver": "adam",
    "tol": 0.0001,
    "validation_fraction": 0.1,
    "verbose": False,
    "warm_start": False,
}

In [35]:
svc = SVC(kernel="linear", probability=True, gamma=0.7, C=1)
svmrbf = SVC(kernel="rbf", probability=True, gamma=0.7, C=1)
mlpc = MLPClassifier(**params)
knc = KNeighborsClassifier(n_neighbors=5)
rfc = RandomForestClassifier(max_depth=3, n_estimators=50)
nb = GaussianNB()
bst = XGBClassifier(
    n_estimators=3, max_depth=3, learning_rate=0.1, objective="binary:logistic"
)
ext = ExtraTreesClassifier(n_estimators=30, max_depth=3)

In [36]:
# Defines the best parameters for NN
config = {
    "l1": 120,
    "l2": 60,
    "batch_size": 8,
    "activation": "elu",
    "epochs": 400,
    "learning_rate": 0.001,
    "loss": "categorical_crossentropy",
    "dropout_rate1": 0.3,
    "dropout_rate2": 0.8,
}

In [37]:
earlystopper = keras.callbacks.EarlyStopping(
    monitor="loss", patience=10, verbose=0, mode="min", restore_best_weights=True
)


batch_size = config.get("batch_size")
epochs = config.get("epochs")

In [38]:
from tensorflow.keras.initializers import HeUniform
from tensorflow.keras.constraints import MaxNorm


def make_model():
    batch_size = config.get("batch_size")
    activation = config.get("activation")
    epochs = config.get("epochs")
    learning_rate = config.get("learning_rate")
    loss = config.get("loss")
    l1 = config.get("l1")
    l2 = config.get("l2")

    dropout_rate1 = config.get("dropout_rate1")
    dropout_rate2 = config.get("dropout_rate2")

    inputs = keras.Input(shape=X_train.shape[1])
    x = keras.layers.UnitNormalization()(inputs)

    x = keras.layers.Dense(
        l1,
        activation=activation,
        kernel_constraint=MaxNorm(3),
        kernel_initializer=HeUniform(),
    )(x)
    x = keras.layers.Dropout(dropout_rate1)(x)

    x = keras.layers.Dense(
        l2,
        activation=activation,
        kernel_constraint=MaxNorm(3),
        kernel_initializer=HeUniform(),
    )(x)
    x = keras.layers.Dropout(dropout_rate2)(x)

    outputs = keras.layers.Dense(6, activation="softmax")(x)

    model = keras.Model(
        inputs=inputs,
        outputs=outputs,
        name="MATseq_NN_multiclass_classifier_v0.3",
    )

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=loss,
        metrics=[
            "accuracy",
            keras.metrics.AUC(name="auc"),
        ],
    )

    return model


nn = make_model()
nn.summary()

In [42]:
eclf = VotingClassifier(
    estimators=[
        ("MLPClassifier", mlpc),
        ("KNeighborsClassifier", knc),
        ("RandomForestClassifier", rfc),
        ("XGBClassifier", bst),
        ("ExtraTreesClassifier", ext),
    ],
    voting="soft",
    n_jobs=-1,
)

In [43]:
models = {
    "LinearSVC": svc,
    "RBF_SVC": svmrbf,
    "MLPClassifier": mlpc,
    "KNeighborsClassifier": knc,
    "RandomForestClassifier": rfc,
    "GaussianNB": nb,
    "XGBClassifier": bst,
    "ExtraTreesClassifier": ext,
    "NeuralNetwork": nn,
    "VotingClassifier": eclf,
}

3. Do leave one out classification X3 with 3 random seeds


In [44]:
keras.backend.clear_session()
keras.utils.set_random_seed(2)

In [45]:
# keras.backend.clear_session()
# keras.utils.set_random_seed(42)

In [46]:
# keras.backend.clear_session()
# keras.utils.set_random_seed(93)

In [47]:
# df_scores = pd.DataFrame(columns=["model","accuracy", "f1", "condition"])

In [66]:
from sklearn.metrics import accuracy_score, f1_score

seeds = [2, 5, 105]
model_names = []
accuracy_scores = []
f1_scores = []
predictions = []


for seed in seeds:
    keras.backend.clear_session()
    keras.utils.set_random_seed(seed)

    for name, model in models.items():
        print(f"Running: {name}")
        if name == "NeuralNetwork":
            model.fit(
                X_train,
                y_train_nn,
                epochs=epochs,
                batch_size=batch_size,
                verbose=True,
                callbacks=[earlystopper],
            )

            train_pred = model.predict(X_train)
            test_pred = model.predict(X_test)

            train_pred = train_pred.argmax(axis=1)
            test_pred = test_pred.argmax(axis=1)
            print(train_pred, test_pred)

        else:
            model.fit(X_train, y_train)

            train_pred = model.predict(X_train)
            test_pred = model.predict(X_test)
            print(train_pred, test_pred)

        # Evaluate model performance
        train_accuracy = accuracy_score(y_train, train_pred)
        test_accuracy = accuracy_score(y_test, test_pred)
        train_f1 = f1_score(y_train, train_pred, average="macro")
        test_f1 = f1_score(y_test, test_pred, average="macro")

        # Print model performance
        print("*" * 50)
        print(f"{name} Training score:")
        print(f"Accuracy: {round(train_accuracy, 4)} | F1 score: {round(train_f1, 4)}")
        print("-" * 20)
        print(f"{name} Validation score:")
        print(f"Accuracy: {round(test_accuracy, 4)} | F1 score: {round(test_f1, 4)}")

        model_names.append(name)
        accuracy_scores.append(test_accuracy)
        f1_scores.append(test_f1)

    # Append model name and metrics to new df
    output = pd.DataFrame()

    condition = "feature_selection"
    # "feature_selection"
    # "no_transform"
    # "SMOT

    output["model"] = model_names
    output["accuracy"] = accuracy_scores
    output["f1"] = f1_scores
    output["condition"] = condition

Running: LinearSVC
[1 5 5 0 4 3 3 4 2 1 2 5 5 0 2 5 5 0 2 4 1 5 4 1 5 5 5 1 0 5 0 2 5 2 4 1 1
 2 0 2 1 4 0 5 3 3 1 3 4 1 4 1 1 4 1 2 4 3 1 4 2 3 0 4 5 2 2 3 1 4 2 0 0 4
 1 1 2 2 2 5 1 1 0 2 1 4 2 0 0 2 4 0 1 3 2 2 1 1 2 4 1 5 2 4 3 5 1 2 5] [3 2 2 1 0 1 0 1 0 1 3 4 2 2 2 4 2 5 5 0 0 1 1 5 0 4 4 2]
**************************************************
LinearSVC Training score:
Accuracy: 1.0 | F1 score: 1.0
--------------------
LinearSVC Validation score:
Accuracy: 0.8929 | F1 score: 0.8937
Running: RBF_SVC
[1 5 5 0 4 3 3 4 2 1 2 5 5 0 2 5 5 0 2 4 1 5 4 1 5 5 5 1 0 5 0 2 5 2 4 1 1
 2 0 2 1 4 0 5 3 3 1 3 4 1 4 1 1 4 1 2 4 3 1 4 2 3 0 4 5 2 2 3 1 4 2 0 0 4
 1 1 2 2 2 5 1 1 0 2 1 4 2 0 0 2 4 0 1 3 2 2 1 1 2 4 1 5 2 4 3 5 1 2 5] [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
**************************************************
RBF_SVC Training score:
Accuracy: 1.0 | F1 score: 1.0
--------------------
RBF_SVC Validation score:
Accuracy: 0.2143 | F1 score: 0.0588
Running: MLPClassifier
[

In [ ]:
df_scores = pd.concat([df_scores, output], ignore_index=True)

In [ ]:
df_scores

In [ ]:
# df_scores = df_scores.drop(df_scores[df_scores['condition'] == 'both_fs_SMOT'].index)

In [ ]:
# df_scores.to_csv("/home/t.afanasyeva/output_MATseq/20240225_scores_classifers/scores.csv")

In [ ]:
result = (
    df_scores.groupby(["model", "condition"]).mean().reset_index().sort_values("f1")
)
result

In [ ]:
# result[result["condition"]=="feature_selection"]["f1"]
# result[result["condition"]=="feature_selection"]["model"]

In [ ]:
# df_scores = df_scores.dropna(axis=0)

In [ ]:
# df_scores =df_scores.rename(columns={"tem": "condition"})

In [ ]:
sns.set_theme(
    context="notebook",
    style="whitegrid",
    palette="deep",
    font="sans-serif",
    font_scale=1,
    color_codes=True,
    rc=None,
)

4.  Measure accuracy, precicion and F1 

In [ ]:
# Set the figure size
plt.figure(figsize=(12, 8))

# plot a bar chart
ax = sns.catplot(
    x=df_scores.model,
    y=df_scores.accuracy,
    kind="bar",
    data=df_scores,
    hue=df_scores.condition,
    estimator=np.mean,
    errorbar="sd",
    capsize=0.2,
    legend=True,
    alpha=0.8,
)


plt.title("Accuracy Scores")
plt.ylim(0, 1)
plt.ylabel("Accuracy")
plt.xlabel("Model")
plt.xticks(rotation=45, ha="right", rotation_mode="anchor")

sns.despine(left=True, right=False, top=True)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 0.9))


plt.tight_layout(w_pad=2, pad=1)
plt.savefig("accuracy.png")
plt.show()

In [ ]:
# Create a barplot for F1 score
plt.figure(figsize=(12, 8))
ax = sns.barplot(
    x=df_scores.model,
    y=df_scores.f1,
    data=df_scores,
    hue=df_scores.condition,
    estimator=np.mean,
    errorbar="sd",
    capsize=0.2,
    legend=True,
    alpha=0.8,
)
plt.title("F1 Scores")
plt.ylim(0, 1)
plt.ylabel("F1 Score")
plt.xlabel("Model")

plt.xticks(rotation=45, ha="right", rotation_mode="anchor")

sns.despine(left=True, right=False, top=True)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1.05, 0.9))


plt.tight_layout(w_pad=2, pad=1)
plt.savefig("f1.png")
plt.show()

In [ ]:
done

4. Try voting classifier of best performing models


In [ ]:
eclf = VotingClassifier(
    estimators=[
        (
            "LinearSVC",
            svc,
        ),
        ("MLPClassifier", mlpc),
    ],
    voting="soft",
    n_jobs=-1,
)

In [ ]:
models = {
    "LinearSVC": svc,
    "MLPClassifier": mlpc,
    "NeuralNetwork": nn,
}

3. Do leave one out classification X3 with 3 random seeds


In [ ]:
# keras.backend.clear_session()
# keras.utils.set_random_seed(2)

In [ ]:
# keras.backend.clear_session()
# keras.utils.set_random_seed(42)

In [ ]:
keras.backend.clear_session()
keras.utils.set_random_seed(93)

In [ ]:
# df_scores = pd.DataFrame(columns=["model","accuracy", "f1", "comment"])

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay


for name, model in models.items():
    print(f"Running: {name}")
    if name == "NeuralNetwork":
        model.fit(
            X_train,
            y_train_nn,
            epochs=epochs,
            batch_size=batch_size,
            verbose=True,
            callbacks=[earlystopper],
        )

        train_pred = model.predict(X_train)
        test_pred = model.predict(X_test)

        train_pred = train_pred.argmax(axis=1)
        test_pred = test_pred.argmax(axis=1)
        print(train_pred, test_pred)

    else:
        model.fit(X_train, y_train)

        train_pred = model.predict(X_train)
        test_pred = model.predict(X_test)
        print(train_pred, test_pred)

    cm = ConfusionMatrixDisplay.from_predictions(
        y_test, test_pred, display_labels=le.classes_, colorbar=False
    )

    cm.plot(cmap="Oranges", values_format="d", xticks_rotation="vertical", ax=None)
    plt.title(f"Confusion Matrix for {name}")

    plt.savefig(f"Confusion Matrix for {name}")
    plt.show()

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

predictions = []
model_names = []
output = pd.DataFrame()

for name, model in models.items():
    if name == "NeuralNetwork":
        test_pred = pd.DataFrame(
            model.predict(X_test),
            columns=list(le.classes_),
            index=list(le.inverse_transform(y_test)),
        )

    else:
        test_pred = pd.DataFrame(
            model.predict_proba(X_test),
            columns=list(le.classes_),
            index=list(le.inverse_transform(y_test)),
        )

    plt.figure(figsize=(12, 8))

    heat = sns.heatmap(test_pred, cmap="YlGnBu")
    # Adjust layout for the plot
    heat.set_title(f"{name}", fontsize=15)
    heat.set_xlabel("Reference", fontsize=15)
    heat.set_ylabel("Sample", fontsize=15, labelpad=10)

    plt.savefig(f"{name}__predictions.png", bbox_inches="tight")
    plt.tight_layout()
    plt.show()

Try with Bacterial samples

In [ ]:
df_path = "/home/t.afanasyeva/MAT_rnaseq/gene_counts/20240224_complete_dataset_274_samples.csv"

bacterial_df = pd.read_csv(df_path, index_col="Unnamed: 0")
bacterial_df_subset = bacterial_df[bacterial_df.index.str.contains("HKSA|HKEB")]

bacterial_labels = []
for i in bacterial_df_subset.index:
    bacterial_labels.append(i.split("_")[2])


y_test = bacterial_labels
bacterial_df_subset.reset_index(drop=True)
X_test = bacterial_df_subset

In [ ]:
df_predictions = pd.DataFrame()

In [ ]:
X_train.shape

In [ ]:
y_train_nn

In [ ]:
X_train.shape

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

model_names = []
predictions = []

for name, model in models.items():
    print(f"Running: {name}")
    if name == "NeuralNetwork":
        model.fit(
            X_train,
            y_train_nn,
            epochs=epochs,
            batch_size=batch_size,
            verbose=True,
            callbacks=[earlystopper],
        )

        train_pred = model.predict(X_train)
        test_pred = model.predict(X_test)

        train_pred = train_pred.argmax(axis=1)
        test_pred = test_pred.argmax(axis=1)
        model_names.append(name)
        predictions.append(list(test_pred))

    else:
        model.fit(X_train, y_train)

        train_pred = model.predict(X_train)
        test_pred = model.predict(X_test)

        model_names.append(name)
        predictions.append(list(test_pred))

    # Append model name and metrics to new df
    output = pd.DataFrame()

    output["model"] = model_names
    output["predictions"] = predictions

Running: LinearSVC
Running: RBF_SVC
Running: MLPClassifier
Running: KNeighborsClassifier
Running: RandomForestClassifier
Running: GaussianNB
Running: XGBClassifier
Running: ExtraTreesClassifier
Running: NeuralNetwork
Epoch 1/400
14/14 [==============================] - 0s 2ms/step - loss: 0.0736 - accuracy: 0.9817 - auc: 0.9996
Epoch 2/400
14/14 [==============================] - 0s 2ms/step - loss: 0.0627 - accuracy: 0.9908 - auc: 0.9999
Epoch 3/400
14/14 [==============================] - 0s 2ms/step - loss: 0.0369 - accuracy: 0.9908 - auc: 1.0000
Epoch 4/400
14/14 [==============================] - 0s 2ms/step - loss: 0.0507 - accuracy: 0.9908 - auc: 0.9999
Epoch 5/400
14/14 [==============================] - 0s 2ms/step - loss: 0.0599 - accuracy: 0.9908 - auc: 0.9999
Epoch 6/400
14/14 [==============================] - 0s 2ms/step - loss: 0.0504 - accuracy: 1.0000 - auc: 1.0000
Epoch 7/400
14/14 [==============================] - 0s 2ms/step - loss: 0.0525 - accuracy: 0.9908 - auc:

In [ ]:
model_names

['LinearSVC',
 'RBF_SVC',
 'MLPClassifier',
 'KNeighborsClassifier',
 'RandomForestClassifier',
 'GaussianNB',
 'XGBClassifier',
 'ExtraTreesClassifier',
 'VotingClassifier']

In [ ]:
list(le.inverse_transform([5, 5, 2, 1, 1, 0, 2, 2, 3, 1, 1, 1]))

In [ ]:
new_pred = []
for i in predictions:
    new_pred.append(list(le.inverse_transform(i)))

In [ ]:
my_dict = dict(zip(model_names, new_pred))

In [ ]:
df = pd.DataFrame.from_dict(my_dict)

In [ ]:
df["True label"] = bacterial_labels

In [ ]:
df

,LinearSVC,RBF_SVC,MLPClassifier,KNeighborsClassifier,RandomForestClassifier,GaussianNB,XGBClassifier,ExtraTreesClassifier,NeuralNetwork,VotingClassifier,True label
0,LPS,IMDM,LPS,LPS,LPS,LPS,LPS,LPS,Pam3,LPS,HKEB
1,LPS,IMDM,LPS,LPS,LPS,LPS,LPS,LPS,Pam3,LPS,HKEB
2,LPS,IMDM,LPS,LPS,LPS,LPS,LPS,LPS,LPS,LPS,HKEB
3,Pam3,IMDM,Pam3,R848,R848,Fla-PA,LPS,R848,Pam3,R848,HKSA
4,Pam3,IMDM,R848,R848,R848,Fla-PA,LPS,R848,Pam3,R848,HKSA
5,Pam3,IMDM,R848,R848,R848,Fla-PA,LPS,R848,Pam3,R848,HKSA
6,Pam3,IMDM,Pam3,Pam3,Pam3,Pam3,Pam3,LPS,Pam3,Pam3,HKEB
7,Pam3,IMDM,Pam3,Pam3,Pam3,Pam3,LPS,LPS,Pam3,Pam3,HKEB
8,Pam3,IMDM,Pam3,Pam3,Pam3,Pam3,LPS,Pam3,Pam3,Pam3,HKEB
9,Pam3,IMDM,Pam3,R848,R848,Fla-PA,Fla-PA,R848,Pam3,R848,HKSA


In [ ]:
df.to_excel("bacterial.xlsx")